# THIS IS JUST A TEST

In [4]:
# Imports

import pystan
import arviz as az
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

In [5]:
# the bioassay.csv data as dictionary
data = {
    'J':4,
    'x':[-0.86,-0.30,-0.05,0.73],
    'N':[5,5,5,5],
    'y':[0,1,3,5]
}

# The stan model 
bioassay_model = """
data {
    int<lower=0> J; // Number of rows in data
    vector[J] x;    // Dose given
    int N[J];    // Amount of trials
    int y[J];    // Deaths
}
parameters {
    vector[2] theta;  // The alpha and beta value in a vector 
}
transformed parameters {
    vector[J] eta = theta[1]+theta[2]*x;
}
model {
    theta ~ multi_normal([0,10], [[4,10],[10,100]]);
    y ~ binomial_logit(N, eta);
}
"""

model = pystan.StanModel(model_code=bioassay_model)
fit = model.sampling(data=data, iter=4000, chains=4)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_018f747e8d0362f5b1db5b6cc8a0732a NOW.


In [6]:
# The data from the sampling
samples = fit.extract(permuted=True)
alpha = samples["theta"][:,0]
beta = samples["theta"][:,1]

# For the Rhat value
print(fit)

Inference for Stan model: anon_model_018f747e8d0362f5b1db5b6cc8a0732a.
4 chains, each with iter=4000; warmup=2000; thin=1; 
post-warmup draws per chain=2000, total post-warmup draws=8000.

           mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
theta[1]   0.98    0.02    0.9  -0.67   0.35   0.93   1.57   2.87   2679    1.0
theta[2]  10.52    0.09   4.57   3.39    7.1   9.96  13.43  20.58   2534    1.0
eta[1]    -8.07    0.06   3.49 -15.89 -10.24  -7.58  -5.49   -2.7   2997    1.0
eta[2]    -2.18    0.02   1.13  -4.69  -2.85  -2.07  -1.37  -0.32   4946    1.0
eta[3]     0.45    0.01   0.79  -1.03  -0.08   0.42   0.96   2.13   3292    1.0
eta[4]     8.66    0.08   3.93   2.41   5.69   8.25  11.14  17.31   2271    1.0
lp__      -7.13    0.02   1.06  -9.84  -7.53  -6.81  -6.38  -6.11   2391    1.0

Samples were drawn using NUTS at Tue Nov 10 12:27:32 2020.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scale reductio